<a href="https://colab.research.google.com/github/Benjamindavid03/MachineLearningLab/blob/main/Decision_Tree_Implementation_Fruit_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Classification
Decision Tree Induction Algorithm for classification

## Dataset - Fruit Dataset.
### Dataset Format: 
1. Each row is an example.
2. The last column is the label.
3. The first two columns are features.
<p> <b>Interesting note:</b> I've written this so the 2nd and 5th examples have the same features, but different labels - so we can see how the tree handles this case.</p>

In [147]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

Column labels.

These are used only to print the tree.

In [148]:
header = ["color", "diameter", "label"]

## Utility classes and functions
Classes and functions that act as utility classes and utility methods

Function : Find the unique values for a column in a dataset

In [149]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])
#######
# Demo:
unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

{'Green', 'Red', 'Yellow'}

Function : Counts the number of each type of example in a dataset

In [150]:
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
#######
# Demo:
class_counts(training_data)
#######

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

Test if a value is numeric for handling diameter from Fruit dataset

In [151]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)
    #######
# Demo:
print(is_numeric(7), is_numeric("Red"))
#######

True False


A Question is used to partition a dataset.

This class just records a 'column number' (e.g., 0 for Color) and a 'column value' (e.g., Green). The 'match' method is used to compare the feature value in an example to the feature value stored in the question.

In [152]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
#######
# Demo:
# Let's write a question for a numeric attribute
Question(1, 3)
# How about one for a categorical attribute
q = Question(0, 'Green')
q

Is color == Green?

In [153]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

True

Partitions a dataset.

For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.

In [154]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
true_rows
# This will contain everything else.
false_rows
#######

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

Calculate the Gini Impurity for a list of rows.

There are a few different ways to do this, I thought this one was
the most concise. See: https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity

In [155]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [156]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [157]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

Information Gain.

The uncertainty of the starting node, minus the weighted impurity of two child nodes.

In [158]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [159]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [160]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [161]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'Red'))

# Here, the true_rows contain only 'Grapes'.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [162]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [163]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[['Green', 3, 'Apple']]

In [164]:
# But, the false-rows are badly mixed up.
false_rows
#######

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

Find the best question to ask by iterating over every feature / value and calculating the information gain

In [165]:
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            print(question, "achieved", gain, "Gain")
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # Fruit dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


#######
# Demo:
# Find the best question to ask first for our Fruit dataset.
best_gain, best_question = find_best_split(training_data)
print("Best question", best_question)
print("Best Gain", best_gain)
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is color == Red? achieved 0.37333333333333324 Gain
Is color == Yellow? achieved 0.17333333333333323 Gain
Is color == Green? achieved 0.1399999999999999 Gain
Is diameter >= 3? achieved 0.37333333333333324 Gain
Best question Is diameter >= 3?
Best Gain 0.37333333333333324


# Decision Tree Implementation

A Leaf node classifies data.

This holds a dictionary of class (e.g., "Apple") -> number of times it appears in the rows from the training data that reach this leaf.

In [166]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

A Decision Node asks a question.

This holds a reference to the question, and to the two child nodes.

In [167]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


## Builds the tree.

Rules of recursion: 
1. Believe that it works. 
2. Start by checking for the base case (no further information gain). 
3. Prepare for giant stack traces

In [168]:
def build_tree(rows):
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)


Function to Print the Decision Tree constructed

In [169]:
def print_tree(node, spacing=""):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


In [170]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is color == Red? achieved 0.37333333333333324 Gain
Is color == Yellow? achieved 0.17333333333333323 Gain
Is color == Green? achieved 0.1399999999999999 Gain
Is diameter >= 3? achieved 0.37333333333333324 Gain
Is color == Yellow? achieved 0.11111111111111116 Gain
Is color == Green? achieved 0.1111111111111111 Gain
Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


Classify function to match and compare with other features then recursively traverse the entire tree

In [171]:
def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [172]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Apple': 1}

In [173]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [174]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Apple': '100%'}

In [175]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Apple': '50%', 'Lemon': '50%'}

## Testing Data to Check the Predictive performance of the Decision Tree classifier

In [176]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

# Evaluating the Decision Tree: 

Get the Actual and the Predicted Instances for the Evaluation set used for testing the Decision Tree classifier

In [177]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}


## References
1. Decision Tree : https://en.wikipedia.org/wiki/Decision_tree
2. Decision Tree Models (Public reserach) : https://github.com/SilverDecisions/SilverDecisions/wiki/Gallery
3. Understanding Decision Trees : https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-understanding-decision-trees-adb2165ccab7
4. Decition Tree Classification : https://towardsdatascience.com/understanding-decision-trees-for-classification-python-9663d683c952
5. Machine Learning : https://github.com/Benjamindavid03/MachineLearningLab
6. Repository : https://github.com/random-forests/tutorials
7. Youtube Video of Gosh : https://www.youtube.com/watch?v=LDRbO9a6XPU
8. Decision Tree without code : https://www.pi.exchange/blog/automate-machine-learning-decision-trees-with-without-code
